In [3]:
from datetime import datetime
import os

from neuroconv.datainterfaces import OpenEphysRecordingInterface

def get_open_ephys_path(rat, date, directory='/Volumes/Tim/Ephys'):
    '''
    Attempt to find OpenEphys recording directory for the given rat and date.
    The assumed directory structure is {directory}/{rat}/{date}/etc
    Complain if no directory is found. 

    Args:
    rat (string): The ID of the rat
    date (string): The date to load, in MMDDYYYY format
    directory (string): The base directory to load from

    Returns:
    string: Path to the OpenEphys recording, or None if not found
    '''
    # Get the path to all data for this session (rat + date)
    ephys_session_path = f"{directory}/{rat}/{date}"
    # Check if the folder exists for this session
    if not os.path.exists(ephys_session_path):
        return f"The folder {ephys_session_path} does not exist."
    
    # Convert date string from MMDDYYYY to YYYY-MM-DD
    # because this is what the OpenEphys recording folder starts with
    date_string = datetime.strptime(date, "%m%d%Y").strftime("%Y-%m-%d")

    # Search for the OpenEphys recording folder starting with the date
    for folder_name in os.listdir(ephys_session_path):
        if folder_name.startswith(date_string):
            return f"{ephys_session_path}/{folder_name}"

rat = 'IM-1478'
date = '07192022'
open_ephys_folder = get_open_ephys_path(rat, date)
print(open_ephys_folder)

# Read Open Ephys data for this session
interface = OpenEphysRecordingInterface(folder_path=open_ephys_folder)
# Extract what metadata we can from the source files
metadata = interface.get_metadata()

print(metadata)


/Volumes/Tim/Ephys/IM-1478/07192022/2022-07-19_13-12-30


/Users/steph/berkelab/jdb_to_nwb/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DeepDict: {'NWBFile': {'session_description': '', 'identifier': '7498d091-3187-499b-a2c3-9bf610190ad9', 'source_script': 'Created using NeuroConv v0.4.11', 'source_script_file_name': '/Users/steph/berkelab/jdb_to_nwb/venv/lib/python3.12/site-packages/neuroconv/basedatainterface.py', 'session_start_time': datetime.datetime(2022, 7, 19, 13, 12, 30)}, 'Ecephys': {'Device': [{'name': 'DeviceEcephys', 'description': 'no description'}], 'ElectrodeGroup': [{'name': 'ElectrodeGroup', 'description': 'no description', 'location': 'unknown', 'device': 'DeviceEcephys'}], 'ElectricalSeries': {'name': 'ElectricalSeries', 'description': 'Acquisition traces for the ElectricalSeries.'}}}


In [28]:
# from neuroconv.utils.dict import load_dict_from_file, dict_deep_update
# metadata_path = "my_lab_metadata.yml"
# metadata_from_yaml = load_dict_from_file(file_path=metadata_path)

# metadata = interface.get_metadata()
# metadata = dict_deep_update(metadata, metadata_from_yaml)

In [29]:
# from pprint import pprint
# pprint(dict(metadata))

{'Ecephys': DeepDict: {'Device': [{'name': 'DeviceEcephys', 'description': 'no description'}, {'name': 'Neuropixels 1.0', 'description': 'My Neuropixels probe'}], 'ElectrodeGroup': [{'name': 'ElectrodeGroup', 'description': 'The whole probe', 'location': 'CA1', 'device': 'Neuropixels 1.0'}], 'ElectricalSeries': {'name': 'ElectricalSeries', 'description': 'Acquisition traces for the ElectricalSeries.'}},
 'NWBFile': DeepDict: {'session_description': '', 'identifier': 'fd89b155-d394-4932-b2ca-be7617ea6f13', 'source_script': 'Created using NeuroConv v0.4.11', 'source_script_file_name': '/Users/rly/mambaforge/envs/test/lib/python3.11/site-packages/neuroconv/basedatainterface.py', 'session_start_time': datetime.datetime(2022, 7, 25, 15, 30)},
 'experimenter': ['Ryan Ly', 'Stephanie Crater'],
 'institution': 'UCSF',
 'session_description': 'my session description',
 'session_start_time': '2024-07-10T20:20:06'}


In [ ]:
# Choose a path for saving the nwb file and run the conversion
nwbfile_path = "output.nwb"  # This should be something like: "./saved_file.nwb"
interface.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata)

In [43]:
from datetime import datetime
from uuid import uuid4

import numpy as np
from dateutil.tz import tzlocal

from pynwb import NWBHDF5IO, NWBFile
from pynwb.ecephys import LFP, ElectricalSeries

In [44]:
nwbfile = NWBFile(
    session_description="my first synthetic recording",
    identifier=str(uuid4()),
    session_start_time=datetime.now(tzlocal()),
    experimenter=[
        "Ryan Ly",
        "Stephanie Crater",
    ],
    lab="Bag End Laboratory",
    institution="University of Middle Earth at the Shire",
    experiment_description="I went on an adventure to reclaim vast treasures.",
    keywords=["ecephys", "exploration", "wanderlust"],
    related_publications="doi:10.1016/j.neuron.2016.12.011",
)

with NWBHDF5IO("output.nwb", "w") as io:
    io.write(nwbfile)


In [41]:
from ruamel.yaml import YAML

with open("my_lab_metadata.yml") as f:
    y = YAML(typ="safe").load(f)
    pprint(y)

    if "experimenter" not in y:
        # warn or provide default value
        pass
    else:
        # ...
        pass
    

{'Ecephys': {'Device': [{'description': 'My Neuropixels probe',
                         'name': 'Neuropixels 1.0'}],
             'ElectrodeGroup': [{'description': 'The whole probe',
                                 'device': 'Neuropixels 1.0',
                                 'location': 'CA1',
                                 'name': 'ElectrodeGroup'}]},
 'institution': 'UCSF',
 'session_description': 'my session description',
 'session_start_time': datetime.datetime(2024, 7, 10, 20, 20, 6)}
